In [1]:
from utils.agents import *
from utils.embedding import *
from utils.json_extract import *
from utils.kg_functions import *
from utils.retrieval import *

### Building a Knowledge Graph from a txt

#### Here, we use the paper as the material, refer to [India's Opportunity to Address Human Resource Challenges](https://pubmed.ncbi.nlm.nih.gov/37448434/#:~:text=The%20paucity%20of%20skilled%20personnel%20must%20be%20addressed,budget%20offers%20an%20unprecedented%20opportunity%20to%20do%20this)

In [2]:
try:
    with open('./prompts/graph.txt', 'r', encoding='utf-8') as file:
        KG_task  = file.read()
except FileNotFoundError:
    res = "File not found."

api_key = "531e9c858c9d4dc1d9295dbf0cdf20e5.zjSejRW3PHh8015f"
agent_KG = Agent(api_key=api_key, model_name="glm-4-long", system_content=KG_task )

In [3]:
file_path = "./datasets/cureus-0015-00000040274.txt"

# Opening and reading the file
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        document_content = file.read()
except FileNotFoundError:
    document_content = "The specified file path was not found."

extracted_content = agent_KG.get_response(user_content=document_content)['response_content']
extracted_json = clean_and_extract_json(extracted_content)

In [4]:
extracted_json

{'nodes': [{'id': 0,
   'name': "India's health indicators",
   'explanation': 'Improved but lag behind peer nations.'},
  {'id': 1,
   'name': 'WHO threshold',
   'explanation': 'Recommended 44.5 doctors, nurses, and midwives per 10,000 population.'},
  {'id': 2,
   'name': 'Public health sector',
   'explanation': 'Underfunded, provides healthcare to vulnerable populations.'},
  {'id': 3,
   'name': 'Private health sector',
   'explanation': 'Dominant provider of healthcare, perceived to provide quality care.'},
  {'id': 4,
   'name': 'National Health Mission (NHM)',
   'explanation': 'Aims to increase healthcare services, focuses on primary healthcare.'},
  {'id': 5,
   'name': 'Accredited Social Health Activists (ASHA)',
   'explanation': 'Female community health workers, serve to increase reach of healthcare.'},
  {'id': 6,
   'name': 'Auxiliary Nurse Midwives (ANM)',
   'explanation': 'Healthcare workers serving a population of 5,000 to 20,000 people.'},
  {'id': 7,
   'name': 'R

### Retrieve From this KG using a query

In [5]:
query = 'How can the National Medical Council (NMC) regulate the growing divide between the public and private health sectors in India, particularly in light of the increasing reliance on Auxiliary Nurse Midwives (ANMs) and Accredited Social Health Activists (ASHAs) for rural healthcare delivery, while addressing the broader challenges of digital technology adoption and health infrastructure development as outlined in the National Health Mission (NHM)?'
beta_1 = 3
alpha_1 = 1
alpha_2 = 2
retreival_res = R(query, extracted_json, alpha_1, alpha_2, beta_1)
retreival_res = json.dumps(retreival_res, ensure_ascii=False, indent=4)

In [6]:
retreival_res

'{\n    "concepts_id/name/explanation_and_similarity_sorted": [\n        {\n            "id": 8,\n            "name": "National Medical Council (NMC)",\n            "explanation": "Replaced Medical Commission, aims to improve regulation of health professionals.",\n            "similarity_score": 0.7361299395796563\n        },\n        {\n            "id": 0,\n            "name": "India\'s health indicators",\n            "explanation": "Improved but lag behind peer nations.",\n            "similarity_score": 0.6073716307823804\n        },\n        {\n            "id": 4,\n            "name": "National Health Mission (NHM)",\n            "explanation": "Aims to increase healthcare services, focuses on primary healthcare.",\n            "similarity_score": 0.7193918994073849\n        },\n        {\n            "id": 5,\n            "name": "Accredited Social Health Activists (ASHA)",\n            "explanation": "Female community health workers, serve to increase reach of healthcare.",\n 

### Construct multi agents provided for users

In [7]:
file_path = './prompts/'
dirs = ['agent1.txt', 'agent2.txt', 'agent3.txt', 'agent4.txt', 'fusion.txt']

# Reading the content of files and storing them in the 'tasks' dictionary directly
tasks = {key: open(f"{file_path}{key}", 'r', encoding='utf-8').read() for key in dirs}

In [9]:
file_path = './prompts/'
dirs = ['agent1.txt', 'agent2.txt', 'agent3.txt', 'agent4.txt', 'fusion.txt']

# Reading the content of files and storing them in the 'tasks' dictionary directly
tasks = {key: open(f"{file_path}{key}", 'r', encoding='utf-8').read() for key in dirs}

# Renaming the keys in the 'tasks' dictionary
tasks = {
    'concepts': tasks['agent1.txt'] + retreival_res,
    'hint': tasks['agent2.txt'] + retreival_res,
    'assess': tasks['agent3.txt'] + retreival_res,
    'questions': tasks['agent4.txt'] + retreival_res,
    'fusion': tasks['fusion.txt']
}

agents = {}
api_key = "531e9c858c9d4dc1d9295dbf0cdf20e5.zjSejRW3PHh8015f"
llms_list = ["glm-4", "glm-4-plus", "glm-4-long", "glm-4-0520", "glm-4-flash"]

# Loop to initialize Agent instances for each LLM and task content, and create a dictionary with LLM+task as keys
for i in llms_list:
    for task_name, task_content in tasks.items():
        key = f"{i}_{task_name}"  # Create a key by combining LLM model name and task name
        agents[key] = Agent(api_key=api_key, model_name=i, system_content=task_content)

#### Testing an Agent

In [11]:
a0 = agents["glm-4-plus_concepts"]
user_input = """
What do  the National Medical Council (NMC) mean?
"""
res0 = a0.get_response(user_content=user_input)['response_content']
print("The reponse is" + " \n" + res0)

The reponse is 
Based on the external knowledge graph provided, here is the explanation for the term "National Medical Council (NMC)":

```json
{
  "term0": {
    "term": "National Medical Council (NMC)",
    "explanation": "Replaced Medical Commission, aims to improve regulation of health professionals."
  }
}
```

The National Medical Council (NMC) is an entity that has replaced the previous Medical Commission. Its primary goal is to enhance the regulation of health professionals, ensuring better standards and practices within the healthcare sector.


### Choose 5 agents

In [13]:
agent1 = agents["glm-4-flash_concepts"]  # Responsible for handling the 'concepts' task using the 'glm-4-flash' model
agent2 = agents["glm-4-plus_hint"]       # Handles the 'hint' task using the 'glm-4-plus' model
agent3 = agents["glm-4-plus_assess"]     # Manages the 'assess' task using the 'glm-4-plus' model
agent4 = agents["glm-4-0520_questions"]  # Processes the 'questions' task using the 'glm-4-0520' model
agent5 = agents["glm-4-flash_fusion"]    # Takes care of the 'fusion' task using the 'glm-4-flash' model

### Generate responses and Fuse them

In [14]:
agents_parallel = [agent1, agent2, agent3, agent4]

response = ""

for agent in agents_parallel:
    res = agent.get_response(user_content=query)['response_content']
    response += res

In [16]:
answer_fusion = agent5.get_response(user_content=response)['response_content']
answer_fusion

'### Response Integration\n\n#### Terms and Their Explanations\n\n1. **National Medical Council (NMC)**: The NMC, which has replaced the Medical Commission, is a regulatory body in India responsible for enhancing the regulation of health professionals. Its aim is to improve the quality and standard of healthcare in the country.\n\n2. **Public and Private Health Sectors**: These sectors within the healthcare system are distinct. The public sector includes government-run hospitals and services, whereas the private sector encompasses healthcare services provided by private entities.\n\n3. **Rural Healthcare Delivery**: This pertains to the provision of healthcare services in rural areas, which often encounter challenges due to geographical isolation, limited resources, and lower access to quality healthcare compared to urban areas.\n\n4. **Auxiliary Nurse Midwives (ANMs)**: ANMs are female community health workers trained to provide primary healthcare services, including maternal and chil

In [17]:
print(answer_fusion)

### Response Integration

#### Terms and Their Explanations

1. **National Medical Council (NMC)**: The NMC, which has replaced the Medical Commission, is a regulatory body in India responsible for enhancing the regulation of health professionals. Its aim is to improve the quality and standard of healthcare in the country.

2. **Public and Private Health Sectors**: These sectors within the healthcare system are distinct. The public sector includes government-run hospitals and services, whereas the private sector encompasses healthcare services provided by private entities.

3. **Rural Healthcare Delivery**: This pertains to the provision of healthcare services in rural areas, which often encounter challenges due to geographical isolation, limited resources, and lower access to quality healthcare compared to urban areas.

4. **Auxiliary Nurse Midwives (ANMs)**: ANMs are female community health workers trained to provide primary healthcare services, including maternal and child health ca

#### The asnwer above is more comprehensive; but more tunning will go on...